In [62]:
import numpy as np
import pandas as pd
dat_dir = "/Users/guoyixuan/Documents/2碩士論文/1_NTPU Thesis/002_Weekly_Report/Weekly_Report_20230924/pca_adaboost/data/"

# test set
orange_20 = pd.read_csv(dat_dir + "orange_20.csv") # 667 20

# train set
orange_80 = pd.read_csv(dat_dir + "orange_80.csv") # 2666 20

# combine train and test to do the data preprocessing
orange = pd.concat([orange_20, orange_80], axis = 0) # 3333 20
orange.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [63]:
# check the variable churn
orange['Churn'].unique()
print(orange['Churn'].value_counts())
# False: 2850
# True: 483
# imbalanced data

# change true/false to 1/0
churn = orange['Churn'][orange['Churn'] == False].count()
orange['Churn'] = np.where(orange['Churn'] == True, 1, 0)

# false
# orange['Churn'] = np.where(orange['Churn'] == 'True', 1, 0)

False    2850
True      483
Name: Churn, dtype: int64


This public dataset is provided by the CrowdAnalytix community as part of their churn prediction competition. The real name of the telecom company is anonymized. It contains 20 predictor variables mostly about customer usage patterns. There are 3333 records in this dataset, out of which 483 customers are churners and the remaining 2850 are non-churners. Thus, the ratio of churners in this dataset is 14%

In [64]:
# preprocessing value of State, Area code, International plan, Voice mail plan and churn
# State
orange['State'] == 'NY'
orange['State'] = orange['State'].astype('category')
#print(orange['State'].cat.categories) # show the categories of State
counts_state = orange['State'].value_counts() # count the number of each category
print(counts_state)
print("Number of state type:{}".format(orange['State'].nunique())) # count the number of unique categories

### meet the high-cardinality feature problem, thus we use target encoding to transform the State will be better
# label encoder to transform the State
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
label_state = le.fit_transform(orange['State'])

# target encoding to transform the State
# pip install category_encoders
# method1 :
import category_encoders as ce
enc = ce.TargetEncoder(cols=['State'])
target = enc.fit_transform(orange['State'], orange['Churn']) # transform the State
# method2 :
orange['state_target'] = orange.groupby('State')['Churn'].transform('mean')
orange[['State', 'state_target', "Churn"]].head()    

WV    106
MN     84
NY     83
AL     80
OR     78
WI     78
OH     78
VA     77
WY     77
CT     74
VT     73
ID     73
MI     73
UT     72
TX     72
IN     71
KS     70
MD     70
NJ     68
NC     68
MT     68
CO     66
NV     66
WA     66
RI     65
MS     65
MA     65
AZ     64
MO     63
FL     63
ND     62
ME     62
NM     62
OK     61
NE     61
DE     61
SC     60
SD     60
KY     59
IL     58
NH     56
AR     55
DC     54
GA     54
HI     53
TN     53
AK     52
LA     51
PA     45
IA     44
CA     34
Name: State, dtype: int64
Number of state type:51


,State,state_target,Churn
0,LA,0.078431,0
1,IN,0.126761,1
2,NY,0.180723,1
3,SC,0.233333,0
4,HI,0.056604,0


In [65]:
# remove area code of the dataframe
del orange['Area code'], orange['State']

# change no/yes to 0/1
orange['International plan'] = np.where(orange['International plan'] == 'Yes', 1, 0)
orange['Voice mail plan'] = np.where(orange['Voice mail plan'] == 'Yes', 1, 0)
# orange['Churn'] = np.where(orange['Churn'] == 'true', 1, 0)
orange.head()
# finally, we get the preprocessed data # 3333 19

,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn,state_target
0,117,0,0,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0,0.078431
1,65,0,0,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1,0.126761
2,161,0,0,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1,0.180723
3,111,0,0,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0,0.233333
4,49,0,0,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0,0.056604


In [71]:
# split the data into train and test
predictors_test = orange.iloc[0:667, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]]
response_test = orange.iloc[0:667, 17]

predictors_train = orange.iloc[667:3333, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]]
response_train = orange.iloc[667:3333, 17]                                                   

In [72]:
response_train.to_csv(dat_dir + "response_train.csv", index = False)
response_test.to_csv(dat_dir + "response_test.csv", index = False)
predictors_train.to_csv(dat_dir + "predictors_train.csv", index = False)
predictors_test.to_csv(dat_dir + "predictors_test.csv", index = False)